In [1]:
# Install required packages (Pathway for streaming, Bokeh for visualization)
!pip install pathway bokeh --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [3]:
# Load the dataset
df = pd.read_csv('/content/dataset.csv')

# Preview the first few rows
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [36]:
import pandas as pd

# Load the original dataset
df = pd.read_csv("dataset.csv")

# Combine date and time columns into a real datetime object
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S',
    errors='coerce'
)

# Drop rows with bad timestamps or critical missing data
df = df.dropna(subset=['Timestamp', 'ID', 'Occupancy', 'Capacity'])

# Save all necessary columns, keeping Timestamp as a datetime object
df[[
    "ID", "SystemCodeNumber", "Timestamp", "Occupancy", "Capacity",
    "QueueLength", "TrafficConditionNearby", "IsSpecialDay",
    "VehicleType", "Latitude", "Longitude"
]].to_csv("parking_stream.csv", index=False)


In [57]:
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00


In [58]:
import pathway as pw

class ParkingSchema(pw.Schema):
    ID: str
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str
    Latitude: float
    Longitude: float


In [43]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)


In [59]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [32]:
window_duration = datetime.timedelta(minutes=30)


# Model 1

In [94]:
daily_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.ID,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_max = pw.reducers.max(pw.this.Occupancy),
        occ_min = pw.reducers.min(pw.this.Occupancy),
        cap = pw.reducers.max(pw.this.Capacity)
    )
    .with_columns(
        price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)


#Model 2

In [113]:
import datetime
import pathway as pw
import pathway.reducers

# Step 1: Parse timestamp and assign lot ID
data_with_time = data.with_columns(
    t=pw.this.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S"),
    lot_id=pw.this.ID
)

# Step 2: Define 30-min tumbling window and reduce to latest record per lot
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.lot_id,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        window_end_t=pw.this._pw_window_end,
        max_t=pw.reducers.max(pw.this.t),
        lot_id=pw.reducers.min(pw.this.ID),
    )
    .join(
        data_with_time,
        pw.this.max_t == data_with_time.t,
        pw.this.lot_id == data_with_time.ID
    )
    .select(
        t=data_with_time.t,
        occ=data_with_time.Occupancy,
        cap=data_with_time.Capacity,
        queue=data_with_time.QueueLength,
        traffic=data_with_time.TrafficConditionNearby,
        is_special=data_with_time.IsSpecialDay,
        vtype=data_with_time.VehicleType,
        lat=data_with_time.Latitude,
        lon=data_with_time.Longitude,
        lot_id=data_with_time.ID
    )
    .with_columns(
    traffic_num = pw.apply(
        lambda traffic: (
            1.0 if traffic == "low" else
            2.0 if traffic == "medium" else
            3.0 if traffic == "high" else
            2.0  # default
        ),
        pw.this.traffic
    ),
    vehicle_weight = pw.apply(
        lambda v: (
            0.5 if v == "bike" else
            1.0 if v == "car" else
            1.5 if v == "truck" else
            1.0  # default
        ),
        pw.this.vtype
    )
)


    .with_columns(
    traffic_num = pw.apply(
        lambda traffic: float(
            1.0 if traffic == "low" else
            2.0 if traffic == "medium" else
            3.0 if traffic == "high" else
            2.0  # default
        ),
        pw.this.traffic
    ),

    vehicle_weight = pw.apply(
        lambda vtype: float(
            0.5 if vtype == "bike" else
            1.0 if vtype == "car" else
            1.5 if vtype == "truck" else
            1.0  # default
        ),
        pw.this.vtype
    )
)

 )


In [95]:
import bokeh.plotting
import panel as pn

# Step 1: Activate Panel extension (once per notebook)
pn.extension()

# Step 2: Define the Bokeh plotting function
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Real-Time Parking Price",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

# Step 3: Bind Pathway stream to the Bokeh figure
viz = delta_window.plot(
    price_plotter,      # rendering function
    sorting_col="t"     # sort by time
)

# Step 4: Create a Panel layout to serve the visualization
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [97]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()